In [1]:
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
df_deal_vec = pd.read_csv('df_new_session_based_window5.csv.gz', compression='gzip')
df_user_vec = pd.read_csv('df_user_sbwin5.csv.gz', compression='gzip')

In [3]:
df_deal_vec = df_deal_vec[['deal_id', 'dealvec']]

In [4]:
df_user_vec = df_user_vec[['account_id', 'uservec']]

In [5]:
df_deal_vec.dealvec = df_deal_vec.dealvec.map(lambda x: x[1:-1].split())

In [6]:
df_user_vec.uservec = df_user_vec.uservec.map(lambda x: x[2:-2].split())

In [7]:
for i in range(200):
    df_deal_vec['dealvec_' + str(i)] = df_deal_vec.dealvec.map(lambda x: float(x[i]))

In [8]:
for i in range(200):
    df_user_vec['uservec_' + str(i)] = df_user_vec.uservec.map(lambda x: float(x[i]))

In [9]:
work_path = ''
train_order = pd.read_csv(work_path + 'train_order_add_negasmpl.gz', compression='gzip')
valid_order = pd.read_csv(work_path + 'valid_order_add_negasmpl.gz', compression='gzip')
test_order = pd.read_csv(work_path + 'test_order_add_negasmpl.gz', compression='gzip')

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
train_order = train_order[['account_id', 'deal_id', 'rating']]
valid_order = valid_order[['account_id', 'deal_id', 'rating']]
test_order = test_order[['account_id', 'deal_id', 'rating']]

In [11]:
df_deal_vec.deal_id = df_deal_vec.deal_id.astype('category').cat.codes.values

In [12]:
df_user_vec.account_id = df_user_vec.account_id.astype('category').cat.codes.values

In [13]:
train_order_after = pd.merge(train_order, df_deal_vec, how='left', left_on=['deal_id'], right_on=['deal_id'])
valid_order_after = pd.merge(valid_order, df_deal_vec, how='left', left_on=['deal_id'], right_on=['deal_id'])
test_order_after = pd.merge(test_order, df_deal_vec, how='left', left_on=['deal_id'], right_on=['deal_id'])

In [14]:
train_order_after = pd.merge(train_order_after, df_user_vec, how='left', left_on=['account_id'], right_on=['account_id'])
valid_order_after = pd.merge(valid_order_after, df_user_vec, how='left', left_on=['account_id'], right_on=['account_id'])
test_order_after = pd.merge(test_order_after, df_user_vec, how='left', left_on=['account_id'], right_on=['account_id'])

In [15]:
train_order_after.shape, valid_order_after.shape, test_order_after.shape

((3165872, 405), (131128, 405), (68402, 405))

In [16]:
train_order_after = train_order_after.dropna()
valid_order_after = valid_order_after.dropna()
test_order_after = test_order_after.dropna()

from sklearn.model_selection import train_test_split
train, test = train_test_split(df_order_after, test_size = 0.1)
train, valid = train_test_split(train, test_size = 0.2)

In [17]:
user_vec_name = []
deal_vec_name = []
for i in range(200):
    user_vec_name.append('uservec_' + str(i))
    deal_vec_name.append('dealvec_' + str(i))

# NN model

In [18]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Flatten, BatchNormalization, Dropout
from keras.losses import binary_crossentropy, mse

In [20]:
user_input = Input(shape=(200,), name='user_vec')
deal_input = Input(shape=(200,), name='deal_vec')
concat = Concatenate()([user_input, deal_input])
fc_0 = Dense(512, activation='relu')(concat)
fc_1 = Dense(256, activation='relu')(fc_0)
#dropout_1 = Dropout(0.2)(fc_1)
fc_2 = Dense(256, activation='relu')(fc_1)
fc_3 = Dense(128, activation='relu')(fc_2)
#dropout_2 = Dropout(0.2)(fc_3)
fc_4 = Dense(64, activation='relu')(fc_3)
fc_5 = Dense(32, activation='relu')(fc_4)
output = Dense(1, activation='sigmoid')(fc_5)

model = Model(inputs = [user_input , deal_input], outputs=output)
model.compile(optimizer='SGD',loss=binary_crossentropy, metrics=['accuracy'])

In [21]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_vec (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
deal_vec (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 400)          0           user_vec[0][0]                   
                                                                 deal_vec[0][0]                   
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 512)          205312      concatenate_2[0][0]              
__________

In [22]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

check_point = ModelCheckpoint('model.hdf5', verbose=True, save_best_only = True)
early_stop = EarlyStopping(patience=5, verbose=True)
model.fit([train_order_after.loc[:, user_vec_name].values, train_order_after.loc[:, deal_vec_name].values], train_order_after.rating, 
          validation_data=([valid_order_after.loc[:, user_vec_name].values, valid_order_after.loc[:, deal_vec_name].values], valid_order_after.rating),
          batch_size=32,
          epochs=100, 
          verbose=True,
         callbacks=[early_stop, check_point])

Train on 2365416 samples, validate on 250 samples
Epoch 1/100
2365416/2365416 [==============================] - 218s 92us/step - loss: 0.5327 - acc: 0.7013 - val_loss: 0.4616 - val_acc: 0.7480

Epoch 00001: val_loss improved from inf to 0.46157, saving model to model.hdf5
Epoch 2/100
2365416/2365416 [==============================] - 218s 92us/step - loss: 0.4431 - acc: 0.7684 - val_loss: 0.4724 - val_acc: 0.7200

Epoch 00002: val_loss did not improve from 0.46157
Epoch 3/100
2365416/2365416 [==============================] - 218s 92us/step - loss: 0.4158 - acc: 0.7850 - val_loss: 0.5602 - val_acc: 0.7320

Epoch 00003: val_loss did not improve from 0.46157
Epoch 4/100
2365416/2365416 [==============================] - 218s 92us/step - loss: 0.3998 - acc: 0.7946 - val_loss: 0.4105 - val_acc: 0.7800

Epoch 00004: val_loss improved from 0.46157 to 0.41054, saving model to model.hdf5
Epoch 5/100
2365416/2365416 [==============================] - 218s 92us/step - loss: 0.3882 - acc: 0.8016

In [24]:
model.load_weights('model.hdf5')
pred_result = model.predict([test_order_after.loc[:, user_vec_name].values, test_order_after.loc[:, deal_vec_name].values])

In [25]:
right_num = 0
for i in range(len(pred_result)):
    if int(pred_result[i][0] >=0.5) == test_order_after.rating.values[i]:
        right_num += 1
print('accuracy: {}'.format((right_num / len(pred_result))))

accuracy: 0.625


#   optimizer  loss  acc (without dropout 6dense layers)
##  SGD crossEntropy  0.84 0.82  0.8380750898693154
##  SGD mse  0.8304177606210149
#   optimizer  loss  acc (with dropout 6dense layers)
##  SGD crossEntropy  